In [1]:
import os, sys,cv2,pickle,pdb,requests,random,time
from sklearn.neural_network import MLPClassifier
import numpy as np
from collections import defaultdict
from tqdm import *
from sklearn.decomposition import PCA
from sklearn.svm import *
from sklearn.linear_model import LogisticRegression
from sklearn import datasets
from sklearn.metrics import f1_score, accuracy_score
from sklearn.model_selection import train_test_split
from functools import wraps
from sklearn.tree import DecisionTreeClassifier
from sys import stderr
import matplotlib.pyplot as plt
from sklearn import tree
from sklearn.decomposition import PCA


## Load CIFAR-10 Data

In [2]:
def load_cifar():
    trn_data, trn_labels, tst_data, tst_labels = [], [], [], []
    def unpickle(file):
        with open(file, 'rb') as fo:
            data = pickle.load(fo, encoding='latin1')
        return data

    for i in trange(1):
        batchName = './data/data_batch_{0}'.format(i + 1)
        unpickled = unpickle(batchName)
        trn_data.extend(unpickled['data'])
        trn_labels.extend(unpickled['labels'])
    unpickled = unpickle('./data/test_batch')
    tst_data.extend(unpickled['data'])
    tst_labels.extend(unpickled['labels'])
    return trn_data, trn_labels, tst_data, tst_labels

## Image preprocessing

In [3]:
def image_prep(image):
    ''' pre-processes the given image
        performs mean normalization and other such operations'''
    return image

## Dimensionality reduction using PCA

In [4]:
def reduce_dim(**kwargs):
    ''' performs dimensionality reduction'''
    if kwargs['method'] == 'pca':
        pass
        
    if kwargs['method'] == 'lda':
        pass

## Classification using kernel SVM

In [5]:
def classify(X, y, **kwargs):
    ''' trains a classifier by taking input features
        and their respective targets and returns the trained model'''
    if kwargs['method'] == 'SVM':
        pass

## Evaluation 

In [6]:
def evaluate(target, predicted):
    f1 = f1_score(target, predicted, average='micro')
    acc = accuracy_score(target, predicted)
    return f1, acc

## Decision Tree

In [7]:
def decision_tree(trn_data, trn_labels, tst_data, tst_labels,depth):
    regr = DecisionTreeClassifier(criterion='gini',splitter='best',max_depth=depth)
    regr.fit(trn_data,trn_labels)
    pred_labels = regr.predict(tst_data)
    return (f1_score(pred_labels, tst_labels, average='micro'),accuracy_score(pred_labels, tst_labels)*100)

## MLP Classifier

In [8]:
def mlp_classifier(trn_data, trn_labels, tst_data, tst_labels):
    clf = MLPClassifier(solver='lbfgs', alpha=1e-5,hidden_layer_sizes=(5, 2), random_state=1)
    clf = clf.fit(trn_data, trn_labels)
    pred_labels = clf.predict(tst_data)
    return (f1_score(pred_labels, tst_labels, average='micro'),accuracy_score(pred_labels, tst_labels)*100)

## Logistic Regression

In [9]:
def logistic_regression(trn_data, trn_labels, tst_data, tst_labels):
    train = LogisticRegression(C=1e6, solver='lbfgs', multi_class='multinomial', max_iter=1000)
    train.fit(trn_data, trn_labels)
    pred_labels = train.predict(tst_data)
    return (f1_score(pred_labels, tst_labels, average='micro'),accuracy_score(pred_labels, tst_labels)*100)

## SVM Kernel

In [10]:
def svm_kernel(trn_data, trn_labels, tst_data, tst_labels,c,iterations):
    svm = SVC(C=c, kernel='rbf', gamma='scale', max_iter=iterations)
    svm.fit(trn_data, trn_labels)
    pred_labels = svm.predict(tst_data)
    accuracy = accuracy_score(pred_labels, tst_labels)
    return (f1_score(pred_labels, tst_labels, average='micro'),accuracy)

## Main Function(Has all 12 functionalities)

In [11]:
def main():
    from sklearn.preprocessing import StandardScaler
    trn_data, trn_labels, tst_data, tst_labels = load_cifar()
    scale = StandardScaler()
    scale.fit(trn_data)
    trn_data, tst_data = list(map(image_prep, trn_data)), list(map(image_prep, tst_data))
    trn_data, tst_data = scale.transform(trn_data), scale.transform(tst_data)
    X_train, X_val, y_train, y_val = train_test_split(trn_data, trn_labels,test_size = 0.20)
    #print('Val - F1 score: {}\n Accuracy: {}'.format(f_score, accuracy_))
    print('The F-1 score and accuracy of logistic regression is: ', logistic_regression(trn_data, trn_labels, tst_data, tst_labels))
    c = 1
    while c < 10:
        for iterations in range(1,1000):
            print('The F-1 score and accuracy of kernel svm is: ', svm_kernel(trn_data, trn_labels, tst_data, tst_labels,c,iterations))
        c += 1
    i = 1
    while i < 10:
        print('The F-1 score and accuracy of decision tree is: ', decision_tree(trn_data, trn_labels, tst_data, tst_labels,i)*100)
        i += 1
    print('The F-1 score and accuracy of mlp classifier is: ', mlp_classifier(trn_data, trn_labels, tst_data, tst_labels))

    #Application of pca on dataset
    i = 1
    while i < 32:
        # varying the dimensions from 1 to 32 we will observe the F1 score and the accuracy of the model.
        pca = PCA(n_components=i)
        pca.fit(trn_data)
        pca_train = pca.fit_transform(trn_data)
        pca_test = pca.fit_transform(tst_data)
        print('The F-1 score and accuracy of logistic regression is: ', logistic_regression(trn_data, trn_labels, tst_data, tst_labels))
        c = 1
        while c < 10:
            for iterations in range(1,1000):
                print('The F-1 score and accuracy of kernel svm is: ', svm_kernel(trn_data, trn_labels, tst_data, tst_labels,c,iterations))
            c += 1
        for i in range(1,10):
             print('The F-1 score and accuracy of decision tree is: ', decision_tree(trn_data, trn_labels, tst_data, tst_labels,i)*100)
        print('The F-1 score and accuracy of mlp classifier is: ', mlp_classifier(trn_data, trn_labels, tst_data, tst_labels))
        i += 1

    # Application of LDA on the test and train data set
    from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
    for no_of_components in range(1,32):
        lda_clf = LDA()
        lda_clf.fit(trn_data,trn_labels)
        train_lda = lda_clf.transform(trn_data)
        test_lda = lda_clf.transform(tst_data)
        print('The F-1 score and accuracy of logistic regression is: ', logistic_regression(trn_data, trn_labels, tst_data, tst_labels))
        i = 1
        while i < 10:
            for iterations in range(1,1000):
                print('The F-1 score and accuracy of kernel svm is: ', svm_kernel(trn_data, trn_labels, tst_data, tst_labels,c,iterations))
            i += 1
        for i in range(1,10):
             print('The F-1 score and accuracy of decision tree is: ', decision_tree(trn_data, trn_labels, tst_data, tst_labels,i)*100)
        print('The F-1 score and accuracy of mlp classifier is: ', mlp_classifier(trn_data, trn_labels, tst_data, tst_labels))

In [13]:
if __name__ == '__main__':
    main()

100%|██████████| 1/1 [00:00<00:00, 45.04it/s]
/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:244: ConvergenceWarning: Solver terminated early (max_iter=1).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)


The F-1 score and accuracy of kernel svm is:  (0.13189999999999999, 0.13189999999999999)


/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:244: ConvergenceWarning: Solver terminated early (max_iter=2).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)


The F-1 score and accuracy of kernel svm is:  (0.11700000000000001, 0.11700000000000001)


/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:244: ConvergenceWarning: Solver terminated early (max_iter=3).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)


...

The F-1 score and accuracy of kernel svm is:  (0.20910000000000001, 0.20910000000000001)


/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:244: ConvergenceWarning: Solver terminated early (max_iter=18).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)


The F-1 score and accuracy of kernel svm is:  (0.19739999999999999, 0.19739999999999999)


/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:244: ConvergenceWarning: Solver terminated early (max_iter=19).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)


KeyboardInterrupt: 